# Exercise solutions

## 1.

(그림 10-3처럼)원래의 인공 뉴런을 사용하여, `A ⊕ B`를 연산해주는 인공 신경망을 그려보자.

   hint : A⊕B = (A⋀$\neg$B)⋁($\neg$A⋀B)

## 2.

일반적으로 분류기능을 하는데 로지스틱 회귀 분류모델을 퍼셉트론(예: 퍼셉트론 학습 알고리즘을 사용하여 학습한 단일 계층의 LTU)보다 더 선호하는 것인가? 어떻게 해야 퍼셉트론이 로지스틱 회귀 분류 모델이랑 성능이 비슷하도록 만들어 줄 수 있는가?

## 3.

첫 MLP의 학습에서 왜 로지스틱 활성화 함수가 핵심이 되는 성분인가?

## 4.

자주 사용이 되는 3가지의 활성화 함수의 이름들을 적어보아라. 그릴수도 있는가?

## 5.

10개의 뉴런들로 이루어진 입력계층과 50개의 뉴런으로 이루어진 은닉계층, 3개의 뉴런을 가진 출력계층들로 만들어진 MLP가 있다고 하자. 또한 모든 인공 뉴런들은 ReLU활성화 함수를 사용한다.
* 입력 행렬 **X**의 크기는 몇인가?
* 은닉 계층에서 편향 벡터 **b**$_h$ 의 크기와 가중치 벡터 **W**$_h$의 크기는 몇인가?
* 출력 계층의 편향 벡터 **b**$_0$ 와 가중치 벡터 **W**$_0$의 크기는 각각 얼마인가?
* 신경망의 출력값 행렬 **Y**의 크기는 몇인가?
* **X**, **b**$_h$, **W**$_h$, **b**$_0$, **W**$_0$에 대한 함수로 신경망 출력 행렬치 **Y**를 연산해주는 수식을 써보아라

## 6.

스팸메일과 그렇지 않은 메일을 구분할 때, 출력 계층에 얼마나 많은 뉴런들이 필요한가? 출력 계층에는 어떤 활성화 함수가 들어가 있어야 하는가? 만약 대신에 MNIST 데이터 세트를 사용한다면, 출력 계층에는 얼마나 많은 뉴런들이 있어야하는가? Chapter2에 있는 집값을 예측하는 신경망을 구축할 때면 출력 계층에는 얼마나 많은 뉴런들이 있어여하며 또 어떤 활성화 함수를 사용해야하는가?

답답

## 7.

역전파 알고리즘은 무엇이고 어떻게 동작하는가? 역전파 알고리즘과 역전모드 autodiff 사이의 차이점은 무엇인가??

## 8.

MLP에 다루어야할 모든 하이퍼파라미터 갯수를 리스트로 만들수 있는가? 만약 MLP가 학습 데이터에 대해서 과잉학습을 한다면, 이를 해결하기 위해서 하이퍼 파라미터들을 어떻게 조절해야하는가??

## 9.

_MNIST 데이터 세트로 심층 MLP를 학습시키고 98%의 precision을 얻을 수 있는지 보자. Chapter9의 마지막 연습문제 처럼, 이것저것 멋대로 붙여보자. (예로, 체크포인트를 저장시켜준다던지, 중단시킬 경우 마지막 체크포인트를 불러온다던지, 요약을 추가해준다던지 텐서보드를 사용해서 그래프를 그려준다던지 등등)_

먼저 심층망을 만들어보자. 한줄이 추가된 것 빼면 이전에 했던것과 정확히 같다. 우리는 학습중에 정확도와 손실함수값을 측정하기 위해 `tf.summary.scalar()`를 추가하고, 우리가 텐서보드를 사용해 학습곡선을 쉽게 보기 좋게 해줄 수 있다.

In [1]:
#필요한 라이브러 호출
import tensorflow as tf
import numpy as np
import os

# 텐서플로우 그래프를 돌리면서 노트북의 결과가 안정적으로 나올 수 있도록 설정
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [2]:
n_inputs = 28*28  # MNIST 특징 값들
n_hidden1 = 300  #첫번째 은닉층의 뉴런수
n_hidden2 = 100  #두번째 은닉층의 뉴런수
n_outputs = 10  #출력 계층 뉴런의 수

In [3]:
reset_graph() # 그래프 초기화

# 입력 뉴런 정의 (플레이스 홀더)
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y") 

In [4]:
# 신경망 정의
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",
                              activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",
                              activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

In [5]:
# 손실함수 연산 정의
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    loss_summary = tf.summary.scalar('log_loss', loss)

In [6]:
#학습률 설정
learning_rate = 0.01

# 학습에 사용할 Optimizer설정
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [7]:
# 가장 높은 점수의 클래스를 출력하여 그 값들을 평균 내주어 정확도를 측정
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)

In [8]:
# 모든 변수 초기화 및 저장 함수 정의
init = tf.global_variables_initializer()
saver = tf.train.Saver()

Now we need to define the directory to write the TensorBoard logs to:

In [9]:
# 현재시간 출력 라이브러리 호출
from datetime import datetime

# 로그파일 생성 함수 정의 - 실행 시간을 이름으로 로그파일 생성
def log_dir(prefix=""):
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_logdir = "tf_logs"
    if prefix:
        prefix += "-"
    name = prefix + "run-" + now
    return "{}/{}/".format(root_logdir, name)

In [10]:
logdir = log_dir("mnist_dnn")

이제 텐서 보드 로그를 작성하는데 사용할 `FileWriter`를 만들어줄 수 있다.

In [11]:
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

왜 초기에 멈추줄 수 있게 하는 것을 구현하지 않았을까? 이를 위해서는 검증 데이터 세트가 필요할 것이다.
다행히, 텐서플로우의 `input_data()`함수가 주는 데이터 세트가 이미 학습 데이터(인스턴스 6만개, 데이터는 이미 섞여있음)와 검증 데이터세트(인스턴스 5000개), 테스트 데이터 세트(인스턴스 5000개)를 나누어 주었다. 그래서 우리는 쉽게 `X_valid`와 `y_valid`를 정의해줄 수 있다.

In [12]:
# 텐서플로우 MNIST 데이터 호출
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [13]:
# 학습 및 테스트 데이터 저장
X_train = mnist.train.images
X_test = mnist.test.images
# 학습 및 테스트 데이터 레이블 값 저장
y_train = mnist.train.labels.astype("int")
y_test = mnist.test.labels.astype("int")

In [14]:
# 검증 데이터 세트 데이터와 레이블 값 저장
X_valid = mnist.validation.images
y_valid = mnist.validation.labels

In [15]:
m, n = X_train.shape

In [16]:
n_epochs = 10001 # 학습 주기 횟수
batch_size = 50 # 배치 사이즈
# 전체 데이터 세트를 배치 사이즈로 나누어준 배치의 개수 
n_batches = int(np.ceil(m / batch_size))

# 체크포인트 저장 경로
checkpoint_path = "/tmp/my_deep_mnist_model.ckpt"
checkpoint_epoch_path = checkpoint_path + ".epoch"
final_model_path = "./my_deep_mnist_model"

best_loss = np.infty
epochs_without_progress = 0
max_epochs_without_progress = 50

with tf.Session() as sess:
    if os.path.isfile(checkpoint_epoch_path):
        # 만약 체크포인트 파일이 존재한다면 모델을 재생성 하고 주기 횟수도 호출해온다.
        with open(checkpoint_epoch_path, "rb") as f:
            start_epoch = int(f.read())
        print("Training was interrupted. Continuing at epoch", start_epoch)
        saver.restore(sess, checkpoint_path)
    else:
        start_epoch = 0
        sess.run(init)

    for epoch in range(start_epoch, n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val, loss_val, accuracy_summary_str, loss_summary_str = sess.run([accuracy, loss, accuracy_summary, loss_summary], feed_dict={X: X_valid, y: y_valid})
        file_writer.add_summary(accuracy_summary_str, epoch)
        file_writer.add_summary(loss_summary_str, epoch)
        if epoch % 5 == 0:
            print("Epoch:", epoch,
                  "\tValidation accuracy: {:.3f}%".format(accuracy_val * 100),
                  "\tLoss: {:.5f}".format(loss_val))
            saver.save(sess, checkpoint_path)
            with open(checkpoint_epoch_path, "wb") as f:
                f.write(b"%d" % (epoch + 1))
            if loss_val < best_loss:
                saver.save(sess, final_model_path)
                best_loss = loss_val
            else:
                epochs_without_progress += 5
                if epochs_without_progress > max_epochs_without_progress:
                    print("Early stopping")
                    break

Epoch: 0 	Validation accuracy: 90.240% 	Loss: 0.35380
Epoch: 5 	Validation accuracy: 95.140% 	Loss: 0.17754
Epoch: 10 	Validation accuracy: 96.580% 	Loss: 0.12690
Epoch: 15 	Validation accuracy: 97.240% 	Loss: 0.10324
Epoch: 20 	Validation accuracy: 97.480% 	Loss: 0.09218
Epoch: 25 	Validation accuracy: 97.680% 	Loss: 0.08200
Epoch: 30 	Validation accuracy: 97.620% 	Loss: 0.07602
Epoch: 35 	Validation accuracy: 97.880% 	Loss: 0.07332
Epoch: 40 	Validation accuracy: 97.820% 	Loss: 0.07050
Epoch: 45 	Validation accuracy: 97.960% 	Loss: 0.06815
Epoch: 50 	Validation accuracy: 98.040% 	Loss: 0.06711
Epoch: 55 	Validation accuracy: 98.020% 	Loss: 0.06707
Epoch: 60 	Validation accuracy: 98.160% 	Loss: 0.06734
Epoch: 65 	Validation accuracy: 98.080% 	Loss: 0.06627
Epoch: 70 	Validation accuracy: 98.220% 	Loss: 0.06721
Epoch: 75 	Validation accuracy: 98.220% 	Loss: 0.06776
Epoch: 80 	Validation accuracy: 98.200% 	Loss: 0.06811
Epoch: 85 	Validation accuracy: 98.140% 	Loss: 0.06694
Epoch: 90 	V

In [17]:
os.remove(checkpoint_epoch_path)

In [18]:
# 학습된 모델 저장 및 테스트 데이터를 사용한 정확도 측정
with tf.Session() as sess:
    saver.restore(sess, final_model_path)
    accuracy_val = accuracy.eval(feed_dict={X: X_test, y: y_test})

INFO:tensorflow:Restoring parameters from ./my_deep_mnist_model


In [19]:
# 정확도 출력
accuracy_val

0.9787